In [29]:
import requests
from bs4 import BeautifulSoup
import tkinter as tk
from tkinter import ttk
import webbrowser

def open_url(event):
    widget = event.widget
    index = widget.tag_names("current")
    if index:
        index = int(index[0])
        if 1 <= index <= len(headlines_with_urls):
            url = headlines_with_urls[index - 1][1]
            webbrowser.open_new(url)

def fetch_news():
    dic={"India":"world/asia/india",
         "Chine":"world/asia/china",
         "Asia":"world/asia",
         "World":"world",
         "Us":"world/us_and_canada",
         "Canada":"world/us_and_canada",
         "Uk":"uk"}
    region = dic[region_var.get()]
    url = f"https://www.bbc.com/news/{region}"

    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    headline_elements = soup.find_all("h3", class_="gs-c-promo-heading__title")

    global headlines_with_urls
    headlines_with_urls = []
    for headline_element in headline_elements:
        headline = headline_element.get_text().strip()
        link = headline_element.find_parent("a")["href"]
        full_link = f"https://www.bbc.com{link}"
        headlines_with_urls.append((headline, full_link))

    news_text.set("Click on a headline to read more:")
    text_widget.config(state="normal")
    text_widget.delete("1.0", "end")

    for index, (headline, _) in enumerate(headlines_with_urls, start=1):
        text_widget.tag_configure(str(index), foreground="blue")
        text_widget.insert("end", f"{index}. {headline}\n", str(index))

    text_widget.config(state="disabled")
    scrollbar.config(command=text_widget.yview)

app = tk.Tk()
app.title("Top News App")

region_var = tk.StringVar()
region_label = ttk.Label(app, text="Select Region:")
region_label.pack()

region_combobox = ttk.Combobox(app, textvariable=region_var, values=["World", "Us", "Uk", "India","Asia","Canada"])
region_combobox.pack()

fetch_button = ttk.Button(app, text="Fetch News", command=fetch_news)
fetch_button.pack()

news_frame = ttk.Frame(app)
news_frame.pack()

scrollbar = ttk.Scrollbar(news_frame)
scrollbar.pack(side="right", fill="y")

text_widget = tk.Text(news_frame, wrap="word", state="disabled", height=15, width=70, cursor="hand2")
text_widget.pack()

text_widget.tag_configure("index", underline=True, foreground="blue")
text_widget.bind("<Button-1>", open_url)
text_widget.config(yscrollcommand=scrollbar.set)

app.mainloop()
